In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sc

In [ ]:
class neuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.inodes = input_nodes
        self.hnodes = hidden_nodes
        self.onodes = output_nodes
        self.lr = learning_rate

        self.wih = np.random.rand(self.inodes, self.hnodes) - 0.5
        self.who = np.random.rand(self.hnodes, self.onodes) - 0.5

        self.activation_function = lambda x: sc.expit(x)

    def train(self, inputs, targets):
        pass

    def query(self, inputs):
        inputs = np.array(inputs, dtype=float, ndmin=2).T
        
        self.hidden_inputs = np.dot(self.wih, inputs)
        self.hidden_outputs = np.expit